# Manipulation des dataframes

##### Importation de pandas et lecture des csvs

In [2]:
import pandas as pd

# répertoire des csvs
cheminCSV = "./../data/dictionary/"

# lecture des 4 fichiers
dfDicoBuch = pd.read_csv(cheminCSV + "1757_Buchanan-J.csv", delimiter=";", encoding="utf8")
dfDicoBigi = pd.read_csv(cheminCSV + "2015_Bigi-B.dict", delimiter=" \[\] ", encoding="utf8", engine='python', names=['motVedette','pronSampa'], skiprows=8 )
BuchananPron = pd.read_csv(cheminCSV + "Buchanan_PronChar_counts.csv", delimiter=";", header = 0, encoding="utf8")
dfMappingSampaIpa = pd.read_csv(cheminCSV + "sampa-ipa.txt", delimiter="  *", encoding="utf8", engine='python', names=['pronSampa','ipa'])


##### Fonctions de conversions

In [3]:
#conversion pour obtenir le mot vedette à partir de la colonne pron
def vieux2recent(mot, dfMapping):
    motV = ""

    for lettre in mot:
        dfRowFinded = dfMapping[dfMapping['char']==lettre]
        if len(dfRowFinded==1):
            motV+=str(dfRowFinded.iloc[0]['graph'])
        else:
            motV+='?'
    return motV

#conversion pour obtenir la pronciation IPA à partir de la Sampa
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""

    listValSampa = strSampa.split(" ")
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['pronSampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

##### Manipulations

In [4]:
#Remplacement des vides
BuchananPron = BuchananPron.fillna("")

In [5]:
#Colonne mot vedette
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: vieux2recent(x['pron'], BuchananPron),1)

In [12]:
#ajout de la prononciation Sampa
#Afin d'avoir un maximum de mots, j'ai utilisé le dictionnaire Bigi qui contient
#beaucoup plus de mots que BigiRichSample.csv.
dfDicoBuch = dfDicoBuch.merge(dfDicoBigi, on="motVedette", how = "left")
dfDicoBuch['pronSampa'] = dfDicoBuch['pronSampa'].fillna("")

pasDansBigi = dfDicoBuch[dfDicoBuch.pronSampa==""]
print(len(pasDansBigi['motVedette']))
print(pasDansBigi)
#5467 mots ne sont tout de même pas présents dans Bigi.
# Il s'agit visiblement de mots qui ne sont plus couramment utilisé au
# vue de l'âge du dictionnaire ou dont l'orthographe a évoluée.

5467

In [7]:
#ajout de la prononciation IPA
dfDicoBuch['ipa'] = dfDicoBuch.apply(lambda x: sampa2ipa(x['pronSampa'], dfMappingSampaIpa),1)

##### Sauvegarde du CSV enrichi

In [8]:
dfDicoBuch.to_csv("./../data/dataframe/BuchananRich.csv", index = False)